In [24]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
from pathlib import Path
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import MultipleLocator
%matplotlib qt5
from astropy.io import fits

In [25]:
base_path = Path('gris_20201120_003/l1_data/') #linux
# base_path = Path('gris_20201120_003\\l1_data') #windows

In [26]:
all_files = base_path.glob('**/*')

In [27]:
files = [file for file in all_files if file.name.endswith('.fits')]

In [28]:
files

[PosixPath('gris_20201120_003/l1_data/gris_20201120_091418_l1p_003_001_0133.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_091657_l1p_003_001_0187.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_090938_l1p_003_001_0038.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_091637_l1p_003_001_0180.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_090926_l1p_003_001_0034.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_091034_l1p_003_001_0057.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_091346_l1p_003_001_0122.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_091406_l1p_003_001_0129.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_090816_l1p_003_001_0010.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_091118_l1p_003_001_0072.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_091619_l1p_003_001_0174.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_091106_l1p_003_001_0068.

In [29]:
files.sort(key=lambda x: int(x.name.split('_')[-1].split('.')[0]))

In [30]:
files

[PosixPath('gris_20201120_003/l1_data/gris_20201120_090749_l1p_003_001_0001.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_090752_l1p_003_001_0002.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_090755_l1p_003_001_0003.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_090758_l1p_003_001_0004.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_090801_l1p_003_001_0005.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_090804_l1p_003_001_0006.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_090807_l1p_003_001_0007.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_090810_l1p_003_001_0008.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_090813_l1p_003_001_0009.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_090816_l1p_003_001_0010.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_090819_l1p_003_001_0011.fits'),
 PosixPath('gris_20201120_003/l1_data/gris_20201120_090822_l1p_003_001_0012.

In [31]:
data = fits.getdata(files[0])
data.shape

(4, 1010, 440, 1)

In [32]:
def get_spectral_image(scannumber, stokes):
#     data, header = sunpy.io.fits.read(files[scannumber])[0]
    data = fits.getdata(files[scannumber])
    if stokes == 0:
        return data[stokes, :, :, 0].T / max_val
    else:
        return data[stokes, :, :, 0].T / data[0, :, :, 0].T

spatial_image = None
max_val = None

def get_spatial_image(wavelength_position):
    global spatial_image, max_val
    if spatial_image is not None:
        return spatial_image[wavelength_position]
#     data, header = sunpy.io.fits.read(files[0])[0]
    data = fits.getdata(files[0])
    spatial_image = np.zeros((data.shape[1], len(files), data.shape[2]), dtype=np.float64)
    for i in range(len(files)):
#         data, header = sunpy.io.fits.read(files[i])[0]
        data = fits.getdata(files[i])
        spatial_image[:, i] = data[0, :, :, 0]
    if max_val is None:
        max_val = spatial_image.max()
    return spatial_image[wavelength_position]

def get_stokes_params(scannumber, slitposition):
#     data, header = sunpy.io.fits.read(files[scannumber])[0]
#     print (data.shape)
    data = fits.getdata(files[scannumber])
    print (data.shape)
    return data[:, :, slitposition, 0] / max_val

In [33]:
size = plt.rcParams['lines.markersize']
wave = np.arange(15640.899, 15640.899 + (1010 * 0.039917), 0.039917)[:-1]
scannumber = 92
slitposition = 301
waveposition = 0
stokesposition = 0

fig, axs = plt.subplots(3, 2, figsize=(10 * 1920/1080, 10))
im01a = axs[0][1].imshow(get_spatial_image(waveposition), cmap='gray', origin='lower')
im00 = axs[0][0].imshow(get_spectral_image(scannumber, stokesposition), cmap='gray', origin='lower')

iquv = get_stokes_params(scannumber, slitposition).astype(np.float64)
for i in range(1, 4):
    iquv[i] = np.round(iquv[i] * 100 / iquv[0], 2)
print (wave.shape, iquv.shape)
im10, = axs[1][0].plot(wave, iquv[0])
im10b = axs[1][0].axvline(wave[waveposition], linestyle='--', color='black', linewidth=0.5)
im11, = axs[1][1].plot(wave, iquv[1])
im20, = axs[2][0].plot(wave, iquv[2])
im21, = axs[2][1].plot(wave, iquv[3])

# axs[1][0].set_ylim(0.35, 0.8)
axs[1][0].set_ylim(np.abs(iquv[0]).min() * 0.95, np.abs(iquv[0]).max() * 1.05)
axs[1][1].set_ylim(-np.abs(iquv[1]).max() * 1.05, np.abs(iquv[1]).max() * 1.05)
axs[2][0].set_ylim(-np.abs(iquv[2]).max() * 1.05, np.abs(iquv[2]).max() * 1.05)
axs[2][1].set_ylim(-np.abs(iquv[3]).max() * 1.05, np.abs(iquv[3]).max() * 1.05)

xx = [slitposition]
yy = [scannumber]

im01b = axs[0][1].scatter(slitposition, scannumber, marker='+', color='red', linewidths=1, s=(size**2) * 8)
im01c, = axs[0][1].plot(np.ones(440) * scannumber, linestyle='--', color='black', linewidth='0.5')

axs[0][0].text(
    -0.22,
    0.5,
    'Stokes',
    transform=axs[0][0].transAxes,
    rotation=90
)

xtick_pos = np.array([15640, 15645, 15650, 15655, 15660, 15665, 15670, 15675, 15680])
axs[0][0].set_xticks((xtick_pos[1:] - 15640.899) / 0.039917, xtick_pos[1:])
axs[0][0].set_yticks(np.array([0, 10, 20, 30, 40, 50])/0.135, [0, 10, 20, 30, 40, 50])
axs[0][1].set_xticks(np.array([0, 10, 20, 30, 40, 50])/0.135, [0, 10, 20, 30, 40, 50])
axs[0][1].set_yticks(np.array([0, 10, 20])/0.135, [0, 10, 20])

axs[0][0].yaxis.set_minor_locator(MultipleLocator(1/0.135))

axs[0][1].xaxis.set_minor_locator(MultipleLocator(1/0.135))
axs[0][1].yaxis.set_minor_locator(MultipleLocator(1/0.135))

axs[1][0].xaxis.set_minor_locator(MultipleLocator(0.5))
axs[1][1].xaxis.set_minor_locator(MultipleLocator(0.5))
axs[2][0].xaxis.set_minor_locator(MultipleLocator(0.5))
axs[2][1].xaxis.set_minor_locator(MultipleLocator(0.5))

axs[0][0].set_ylabel(r'x [arcsec]')
axs[0][1].set_xlabel(r'x [arcsec]')
axs[0][1].set_ylabel(r'y [arcsec]')
axs[1][0].set_ylabel(r'$I/I_{c}$')
axs[1][1].set_ylabel(r'$Q/I$ [%]')
axs[2][0].set_ylabel(r'$U/I$ [%]')
axs[2][1].set_ylabel(r'$V/I$ [%]')
axs[2][0].set_xlabel(r'Wavelength [$\mathrm{\AA}$]')
axs[2][1].set_xlabel(r'Wavelength [$\mathrm{\AA}$]')

stokes_axs = plt.axes([0.03, 0.685, 0.03, 0.25])
scan_axs = plt.axes([0.13, 0.96, 0.3, 0.03])
wave_axs = plt.axes([0.6, 0.96, 0.25, 0.03])

def update_stokes(val):
    global scannumber, stokesposition
    stokesposition = val
    dd = get_spectral_image(scannumber, stokesposition)
    im00.set_data(dd)
    im00.set_clim(dd.min(), dd.max())
    fig.canvas.draw_idle()

def update_scan(val):
    global scannumber, stokesposition
    scannumber = val
    dd = get_spectral_image(scannumber, stokesposition)
    im00.set_data(dd)
    im00.set_clim(dd.min(), dd.max())
    im01c.set_ydata(np.ones(440) * scannumber)
    fig.canvas.draw_idle()

def update_wave(val):
    global waveposition
    waveposition = np.argmin(np.abs(wave - val))
    dd = get_spatial_image(waveposition)
    im01a.set_data(dd)
    im01a.set_clim(dd.min(), dd.max())
    im10b.set_xdata(wave[waveposition])
    fig.canvas.draw_idle()

def on_click_event(event):
    global scannumber, slitposition, xx, yy
    ax = event.inaxes

    if ax is None or ax != axs[0][1]:
        return
    
    slitposition = np.round(event.xdata, 0).astype(np.int64)
    scannumber = np.round(event.ydata, 0).astype(np.int64)

    iquv = get_stokes_params(scannumber, slitposition).astype(np.float64)
    for i in range(1, 4):
        iquv[i] = np.round(iquv[i] * 100 / iquv[0], 2)

    im10.set_ydata(iquv[0])
    im11.set_ydata(iquv[1])
    im20.set_ydata(iquv[2])
    im21.set_ydata(iquv[3])
    
    axs[1][0].set_ylim(np.abs(iquv[0]).min() * 0.95, np.abs(iquv[0]).max() * 1.05)
    axs[1][1].set_ylim(-np.abs(iquv[1]).max() * 1.05, np.abs(iquv[1]).max() * 1.05)
    axs[2][0].set_ylim(-np.abs(iquv[2]).max() * 1.05, np.abs(iquv[2]).max() * 1.05)
    axs[2][1].set_ylim(-np.abs(iquv[3]).max() * 1.05, np.abs(iquv[3]).max() * 1.05)
    
    xx = [slitposition]
    yy = [scannumber]
    im01b.set_offsets(np.c_[xx,yy])

    fig.canvas.draw_idle()

stokes_slider = Slider(
    ax=stokes_axs,
    label='',
    valmin=0,
    valmax=3,
    valinit=0,
    valstep=1,
    orientation='vertical'
)
scan_slider = Slider(
    ax=scan_axs,
    label='',
    valmin=0,
    valmax=199,
    valinit=scannumber,
    valstep=1
)
wave_slider = Slider(
    ax=wave_axs,
    label='',
    valmin=15640.899,
    valmax=15681.175253,
    valinit=15640.899,
    valstep=0.039917
)
stokes_slider.on_changed(update_stokes)
scan_slider.on_changed(update_scan)
wave_slider.on_changed(update_wave)
fig.canvas.mpl_connect('button_press_event', on_click_event)

plt.subplots_adjust(left=0.1, bottom=0.05, right=0.99, top=0.95, wspace=0.2, hspace=0.2)
plt.show()

(4, 1010, 440, 1)
(1010,) (4, 1010)


In [34]:
splitting = 15648.448-15647.625

In [35]:
mag_field = splitting / (4.67e-13 * 15648.514**2)

In [36]:
mag_field

7196.755512437031

(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)
(4, 1010, 440, 1)


: 